In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 14.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://beelike-winkingly-terra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://beelike-winkingly-terra.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學 (Ming Hsin University of Science and Technology, 簡稱明新科大) 位於臺灣新竹縣，是一所歷史悠久、以務實致用為導向的私立科技大學。

**以下是明新科技大學的簡要介紹：**

1.  **創校與沿革：** 學校創立於1966年，前身為「明新工業專科學校」，在臺灣技職教育體系中擁有深厚基礎。後於2002年成功升格為科技大學，持續轉型與發展。

2.  **地理優勢：** 明新科大最大的特色之一是其優越的地理位置，緊鄰全球聞名的新竹科學園區。這使得學校與高科技產業之間的連結非常緊密，學生擁有豐富的實習機會，畢業生也具備高度的就業競爭力，深受產業歡迎。

3.  **教育理念與特色：**
    *   **務實致用：** 學校以「誠、樸、精、勤」為校訓，致力於培育具備專業技能、實務能力與職業道德的產業人才，強調學用合一、理論與實務並重。
    *   **產學合作：** 明新科大在產學合作方面成果豐碩，積極與企業界合作開設課程、共同研發，為學生提供與產業接軌的學習環境。
    *   **職涯發展：** 學校非常重視學生的職涯發展，提供多元的輔導與資源，協助學生順利進入職場。

4.  **學術組織：** 明新科大目前設有四大教學單位，提供多樣化的學士、碩士及進修學位課程：
    *   **工程學院：** 涵蓋電子、電機、機械、土木等傳統與新興工程領域。
    *   **管理學院：** 包含企業管理、資訊管理、行銷與流通管理等專業。
    *   **服務事業與設計學院：** 發展觀光、旅館、餐飲、幼保以及多媒體與遊戲設計等領域。
    *   **人文社會學院：** 提供基礎教育與相關人文社會科學課程。

5.  **發展願景：** 明新科大不僅是桃竹苗地區重要的技職人才培育基地，也積極拓展國際合作，提升學術視野與全球競爭力，旨在成為一所具備產業影響力、培養跨領域創新人才的卓越科技大學。

總體而言，明新科技大學是一所以培育產業所需實務人才為核心，並積極與在地產業接軌，提供學生優質學習與職涯發展機會的現代化科技大學。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是 **劉國偉 博士**。

他於2022年8月1日就任。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:02:08] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[]}
BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594449234427118167","quoteToken":"N_QVdvKJRgDDwmmuif3WLRt1PkpLP3hP-eert2Sn07Xe4IHKwo2hNmF4No5k0WrfEo_HMj4het2Vid1AGUKJ_pc0eYkA0oICGBZ9JDV5zeq90iPecIMvG33U_6RCp85ffdwm077QrlRC8BxwGKUQ4Q","markAsReadToken":"MJUJ7Ly2gm-I2RCtuW-y7fjTbpOAI7WbBzedqP6CrljU6_fIJYV78ghFneIydwwNrbihvx8dLtp93W2PgXMunm2xat_cRTs6jytEhXoB87RHkDU0bxn7DPlx9A7XlvYkCoK6tyVn8CVS51b9Jy8SpGADQQdP8rOfGYQbb-qaUDjj3Kt-kgelpyDmHDqApSIqYSVf73y_GR2UWBgzXYISnw","text":"AI 簡介明新科技大學"},"webhookEventId":"01KDS669GT9SXBDR3K9MTGRBM4","deliveryContext":{"isRedelivery":false},"timestamp":1767150658590,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"6e463eccc627467d9b43e9b25c1befa9","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 03:11:08] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594455428608295548","quoteToken":"Kph2QtIODVTJE686nl6aK-g1mGMOmDcJSedHVTlza81YDY6n_rAFu7iOMOj3O9KEvLZ-Gz8oSxF9J0WcQGZDqRbg9XVslOLz3C7I0L0lpMKmTuI4KAqMutVjBB70pwRoWuy3y_SF0yE-Kp_4Yj_ziQ","markAsReadToken":"-nhVx2_5F4DnYqJduni6NZISZ2MaJT4hnoInqiHCjRw4_a9JE-L4UfG59zplTudLvIX9M1JplpPt0AwPDdsTch8rEA0ZcfawTvq7EDAGoDTkevhAJA1owTgjCm4XeBOx2BxICsyKRbFUopGkIG0I-2SN2WKqvBMXtuAF_9g6tqDgUi2V3lgs2oGxD6hPekBoaMAAhaHRY4mVtuXoCW4tCw","text":"AI 介紹袋鼠"},"webhookEventId":"01KDS9PYZPCDBCF9YB809QRPA2","deliveryContext":{"isRedelivery":false},"timestamp":1767154350586,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"06f62eeec88c4480982787c237b0913e","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:12:44] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"text","id":"594455523349496015","quoteToken":"GlMYkCcPHXfZpCwPok38uQvtbUmfnj6AuhRJ1iOfdNXhKznhBLl4-CLaGDB0_5AK8pP3dKalbmogE38dGcqDJVKblS-ZgSBM32eAnPjSmAC1ygDuVdj6gb4xZBGUd9dKqtMd5UxIRexSJOl1_kVrvg","markAsReadToken":"9QLyBiNTEOaKgyylwopsPztwpz1zy0j25U9av9Uu7cNMykCKb7vRwcothroJ9CTR9E-PB2fPQehu9pG3MFSoMgVDFGx_kGXSDbgTEU9FDJnCtK6PFHJI4f6OmxTpKyP8HmqIx4G1RsN8uHdUDkIkrq4pHSpSXa_5IztnC-MXg1zJfsYXrZ-201i5NL3-0zoWeIZSDvx8Dqju3QM1vHx_Pg","text":"AI 介紹河狸"},"webhookEventId":"01KDS9RP11K88650QDREMFR6TH","deliveryContext":{"isRedelivery":false},"timestamp":1767154407224,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"bb7e5d432d604d42bbfc87b46e25958c","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:13:40] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:16:09] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ud89d0d522afbfeb8be5fbbbf47121322","events":[{"type":"message","message":{"type":"file","id":"594455795140133086","markAsReadToken":"RCOI7Vc8nLPxBge-85gdk-1nKJF--Ic-DZ0Crj3_7aGRPcO0erZOOfJM2P7oLIeC5xljM5WhpeGmzWRe4-aIJrfDA-GTz3624-Qz-bwPwndAJ_jxBZiWzlGPctlG7RwcGOw9oFQsEjKF1x1rhSKJrJxaqcqtVx0w5ufifCKQ_pXprCZ_tclJEq2zwzGf0B9uMxfc9fq07kO_Y29U8Cj4Kw","fileName":"FAQ.txt","fileSize":24,"contentProvider":{"type":"line"}},"webhookEventId":"01KDS9XM1A6FS33XA18Z16ZN3P","deliveryContext":{"isRedelivery":false},"timestamp":1767154569117,"source":{"type":"user","userId":"Ufbcafa7664d78087c8c094f2128b40c6"},"replyToken":"4fe6d2f127c14e4ab7303c1345c2246e","mode":"active"}]}
